In [ ]:
import csv
import json
import math
import numpy as np

import os

### Create Rotation Matrix from Orientation Angles (Head, Pitch, Roll)

In [ ]:
def make_rotation_matrix(head, pitch, roll):
    '''
    Create a rotation matrix based on head, pitch and roll angles that can be used to get the orientation of the whale in 3 dimensions.
    '''
    return np.array([
        [math.cos(head) * math.cos(pitch), -1 * math.cos(head) * math.sin(pitch) * math.sin(roll) - math.sin(head) * math.cos(roll), -1 * math.cos(head) * math.sin(pitch) * math.cos(roll) + math.sin(head) * math.sin(roll)],
        [math.sin(head) * math.cos(pitch), -1 * math.sin(head) * math.sin(pitch) * math.sin(roll) + math.cos(head) * math.cos(roll), -1 * math.sin(head) * math.sin(pitch) * math.cos(roll) - math.cos(head) * math.sin(roll)],
        [math.sin(pitch), math.cos(pitch) * math.sin(roll), math.cos(pitch) * math.cos(roll)]])

In [ ]:
def get_orientation(orientation):
    '''
    Get the orientation of the whale in 3 dimensions.
    '''
    return np.array([orientation['x'], orientation['y'], orientation['z']])

In [ ]:
def get_orientation_angles(orientation):
    '''
    Get the orientation angles of the whale in 3 dimensions.
    '''
    return np.array([orientation['head'], orientation['pitch'], orientation['roll']])

In [ ]:
# Gets the trajectory estimates for the whale, used to create all of the different movement plots.
import math
import numpy as np
import scipy.io
import pandas as pd

def create_whale_trajectory(prh_file):
    # Create the whale trajectories from the PRH file, using rotation matrix to get direction angle.
    whale_data = scipy.io.loadmat(prh_file)

    print("Whale PRH Data: ", pd.DataFrame(whale_data).head(5))

    name = prh_file.split('/')[-1]
    frame_rate = whale_data['fs'][0][0]

    x = [0]
    y = [0]
    z = [0]
    speed_list = [0]

    if 'head' in whale_data.keys(): # This just makes sure that the head, pitch and roll angles have been computed. It seems like with the new data you will have to compute these.
        for i in range(len(whale_data['head'])):
            rotation_matrix = make_rotation_matrix(whale_data['head'][i][0], whale_data['pitch'][i][0], whale_data['roll'][i][0])
            no_speed_whale = np.array([1, 0, 0])
            rotated_whale = np.matmul(no_speed_whale, rotation_matrix)
            if i != len(whale_data['head']) - 1 and abs(math.atan(rotated_whale[2]/(math.sqrt(rotated_whale[0]**2 + rotated_whale[1]**2)))) > math.pi/6:
                # This checks that the angle is big enough for a dive, and then computes the speed from change in depth.
                speed_multiplier = abs(whale_data['p'][i][0] - whale_data['p'][i+1][0])/rotated_whale[2]
            else:
                # Otherwise we just assume the whale moves at 1.5m/s at the surface. This was given by a biologist but can be changed if incorrect.
                speed_multiplier = 1.5/frame_rate
            speed_list.append(speed_multiplier)
            whale = speed_multiplier * rotated_whale
            x.append(x[-1] + whale[0])
            y.append(y[-1] + whale[1])
            z.append(-1 * whale_data['p'][i][0])
            speed_list.append(speed_multiplier)
        pitch = whale_data['pitch']
        roll = whale_data['roll']
        head = whale_data['head']
        acc = whale_data['Aw']
    else:
        raise ValueError('Need to implement transformation when not given pitch, roll and head explicitly.')

    return {'name': name, 'x': x, 'y': y, 'z': z, 'fs': frame_rate, 'pitch': pitch, 'roll': roll, 'head': head, 'color': 'black', 'speed': speed_list}